When parsing a decorator in source code, Python takes the decorated
function and passes it as the first argument to the decorator function. So
how do you make a decorator accept other arguments? The answer is: make
a decorator factory that takes those arguments and returns a decorator,which is then applied to the function to be decorated.

Example 9-21. Abridged registration.py module from Example 9-2, repeated
here for convenience

In [1]:
registery = []

def register(func):
    print(f'runninh register({func})')
    registery.append(func)
    return func

@register
def f1():
    print('running f1()')

print('running main()')
print('registery ->', registery)
f1()


runninh register(<function f1 at 0x7f9a662ee830>)
running main()
registery -> [<function f1 at 0x7f9a662ee830>]
running f1()


Example 9-22. To accept parameters, the new register decorator must be
called as a function

In [6]:
registery = set()

def register(active=True):
    def decorate(func):
        print('running register'
              f'(active={active})->decorate({func})')
        if active:
            registery.add(func)
        else:
            registery.discard(func)
        return func
    return decorate

@register(active=False)
def f1():
    print('running f1()')

@register()
def f2():
    print('running f2()')

def f3():
    print('running f3()')



running register(active=False)->decorate(<function f1 at 0x7f9a3fb19240>)
running register(active=True)->decorate(<function f2 at 0x7f9a3fb19480>)


In [7]:
registery

{<function __main__.f2()>}

Example 9-24. Module clockdeco_param.py: the parameterized clock
decorator

In [8]:
import time

DEFAULT_EMT = '[{elapsed:0.8f}s] {name}({args}) -> {result}'

def clock(fmt=DEFAULT_EMT):
    def decorate(func):
        def clocked(*_args):
            t0 = time.perf_counter()
            _result = func(*_args)
            elapsed = time.perf_counter() - t0
            name = func.__name__
            args = ', '.join(repr(arg) for arg in _args)
            result = repr(_result)
            print(fmt.format(**locals()))
            return _result
        return clocked
    return decorate

if __name__ == '__main__':

    @clock()
    def snooze(seconds):
        time.sleep(seconds)

    for i in range(3):
        snooze(.123)
        

[0.12318837s] snooze(0.123) -> None
[0.12333113s] snooze(0.123) -> None
[0.12318887s] snooze(0.123) -> None


Example 9-27. Module clockdeco_cls.py: parameterized clock decorator
implemented as class

In [10]:
import time

DEFAULT_FMT = '[{elapsed:0.8f}s] {name}({args}) -> {result}'

class clock:
    def __init__(self, fmt=DEFAULT_FMT):
        self.fmt = fmt

    def __call__(self, func):
        def clocked(*_args):
            t0 = time.perf_counter()
            _result = func(*_args)
            elapsed = time.perf_counter() - t0
            name = func.__name__
            args = ', '.join(repr(arg) for arg in _args)
            result = repr(_result)
            print(self.fmt.format(**locals()))
            return _result
        return clocked

In [11]:
if __name__ == '__main__':

    @clock()
    def snooze(seconds):
        time.sleep(seconds)

    for i in range(3):
        snooze(.123)

[0.12315655s] snooze(0.123) -> None
[0.12317633s] snooze(0.123) -> None
[0.12317874s] snooze(0.123) -> None
